# Getting data together


In [14]:
# Importing dependencies 
import pandas as pd
import requests
import json
import time
import numpy as np


In [2]:
# URL for our query, built with the filter/export tool in the CDC website, super helpful.

url = "https://data.cdc.gov/resource/xkb8-kh2a.json?$query=SELECT%0A%20%20%60state%60%2C%0A%20%20%60year%60%2C%0A%20%20%60month%60%2C%0A%20%20%60period%60%2C%0A%20%20%60indicator%60%2C%0A%20%20%60data_value%60%2C%0A%20%20%60percent_complete%60%2C%0A%20%20%60percent_pending_investigation%60%2C%0A%20%20%60state_name%60%2C%0A%20%20%60footnote%60%2C%0A%20%20%60footnote_symbol%60%2C%0A%20%20%60predicted_value%60%0AWHERE%0A%20%20(%60indicator%60%20%3D%20%22Number%20of%20Drug%20Overdose%20Deaths%22)%0A%20%20AND%20((%60year%60%20IN%20(2021%2C%202022%2C%202023))%0A%20%20%20%20%20%20%20%20%20AND%20caseless_one_of(%60month%60%2C%20%22December%22))"
drug_overdoses_data = []

In [3]:
#looping through all the years in our list to fetch the corresponding data

response = requests.get(url)
response_json = response.json()

for x in response_json:
    info = {}
    info["Year"] = x["year"]
    info["State name"] = x["state_name"]
    info["Deaths by overdose"] = np.NaN
    if "data_value" in x :
        info["Deaths by overdose"] = x["data_value"]
    drug_overdoses_data.append(info)


time.sleep(1)


In [4]:
#Creates the data frame from the drug_overdose_data list
overdoses_df = pd.DataFrame(drug_overdoses_data)

overdoses_df.head()


,Year,State name,Deaths by overdose
0,2021,Alaska,255
1,2022,Alaska,247
2,2023,Alaska,357
3,2021,Alabama,1333
4,2022,Alabama,1452


In [5]:
# Export the overdoses_df into a csv
overdoses_df.to_csv("overdoses_df.csv")


In [13]:
# Read saved data
overdoses_df = pd.read_csv("overdoses_df.csv",index_col=0)

# Display sample data to check it looks ok
overdoses_df.head()

,Year,State name,Deaths by overdose
0,2021,Alaska,255
1,2022,Alaska,247
2,2023,Alaska,357
3,2021,Alabama,1333
4,2022,Alabama,1452


In [9]:
# The CDC includes New Yor City numbers separately from New York State

overdoses_df = overdoses_df.loc[overdoses_df["State name"] != "New York City",:]

overdoses_df.count()

Year                  156
State name            156
Deaths by overdose    156
dtype: int64

In [12]:
#Groupong by state and calculating the yearly average deaths
average_deaths = overdoses_df.groupby("State name")["Deaths by overdose"].mean()

average_deaths_df = average_deaths.reset_index()
average_deaths_df.columns= ["State Name", "Average Deaths by OD"]

average_deaths_df.head()



,State Name,Average Deaths by OD
0,Alabama,1461.000000
1,Alaska,286.333333
2,Arizona,2745.666667
3,Arkansas,578.333333
4,California,11842.000000


In [ ]:
# Export the Yearly OD death average by state into a csv
average_deaths_df.to_csv("average_deaths_df.csv")

